<a href="https://colab.research.google.com/github/XavierCachan/moduleIA_S4/blob/main/Neurone_artificiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Retour vers la partie 2 : Caractérisation d'une batterie par optimisation](https://colab.research.google.com/drive/1JXGgboVAfwCTm1tow6axvQS3vtgH3vH3#scrollTo=4B-_OgfjO2Ab)

# **TP IA Partie 3 - Caractérisation d'une batterie par un neurone artificiel - IUT de Cachan GEII2 2024**
XM - Février 2024 - Version : 0.5

-----

Note : Pour avancer dans ce notebook, il suffit d'exécuter (petite flèche), ou compléter puis exécuter, les différents blocs de code placés ci-dessous.

Pour cette troisième partie, nous allons traiter le même problème de carctérisation de batterie que dans la partie 2, mais cette fois en utilisant les modules Python permettant de créer des neurones artificiels. Le modèle recherché est donc toujours **U_bat = E - Ri * I_bat**  

**1. Import des modules nécessaires pour les calculs et l'affichage**

In [ ]:
import pandas as pd               # Module pour la construction d'un réseau de neurones
import tensorflow as tf           # TensorFlow est LE module utilisé pour l'entrainement d'un réseau de neurones
import numpy as np                # Module de fonctions mathématiques
import matplotlib.pyplot as plt   # Module pour l'affichage

**2. Fabrication des signaux de mesure de tension U_mes (ce sont les mêmes codes que dans la Partie 2)**

In [ ]:
N = 40                           # Nombre de mesures
I_bat = np.linspace(0,10,num=N)  # Fabrication d'un vecteur de N points régulièrement espacés entre 0 et 10 (0, 0.25, 0.5,...9.75 Ampères)

# Définition de la fonction mathématique théorique. Le modèle que l'on va essayer de retrouver automatiquement est U_bat = E - Ri * I_bat
Ri_th = 0.15                   # Poids théorique à trouver. Par défaut 0.15 ohms
E_th = 12                      # Biais théorique à trouver. Par défaut 12 V
U_th = E_th - Ri_th * I_bat     # Valeurs des tensions si la mesure est "parfaite"

Pour rendre le cas plus réel, on ajoute du bruit de mesure

In [ ]:
sigma = 0.3             # "Intensité" de bruit
bruit = sigma*(np.random.randn(len(U_th)))
U_mes = U_th + bruit    # Signaux de mesure "réels" = signaux parfaits + bruit

In [ ]:
# Affichage des mesures
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (I_bat, U_mes, marker="+", linestyle="none", linewidth=1, label="Tension réelle mesurée")
ax.plot (I_bat, U_th, linestyle="--", linewidth=0.5, label="Droite parfaite (ce que l'on souhaite retrouver)")
ax.set_title("Mesures sur la batterie")
ax.set_xlabel("Entrées Courant I_bat")
ax.set_ylabel("Tension U_bat")
ax.legend()
plt.show()

**3. Recherche des paramètres de la batterie avec un neurone**

Un neurone artificiel (voir dessin ci-dessous) associe des poids w à chacune de ses entrées x (=caractéristiques), et ajoute un biais b. On peut ensuite ajouter une fonction d'activation f pour obtenir la sortie y (=étiquette). Si l'on prend une seule entrée, et pas de fonction d'activation, on retrouve le y = w * x + b de la partie 2 !

<div>
<img src="https://user.oc-static.com/upload/2018/12/10/15444553183515_neuroneformel-1.png" width="500"/>
</div>


Pour travailler avec un réseau de neurones, il faut commencer par écrire **une fonction pour définir le réseau** : nombre de couches, type de couche, nombre de neurones par couches, ... Ici pour cette première approche un seul neurone est suffisant pour retrouver une équation y = w * x + b pour laquelle on espère obtenir après entrainement w = -Ri_th et b = E_th.

In [ ]:
# Fonction pour contruire un modèle de régression linéaire
def construction(taux_apprentissage):
  modele = tf.keras.models.Sequential()  # Définition d'un modèle de réseau séquentiel (= ensemble de couches)

  # Description du réseau : ici 1 couche de 1 neurone, et pas de fonction d'activation
  # Une couche "Dense" signifie une couche de neurones artificiels
  # D'autres types de couches sont possibles : layers.Rescaling(), layers.Conv2D(), layers.MaxPooling2D(), layer.Dropout()... comme vu dans le cours
  modele.add(tf.keras.layers.Dense(units=1,             # unit = le nombre de sorties de la couche, donc le nombre de neurones
                                  input_shape=(1,)))    # 1 car la couche n'a qu'une seule entrée x (une valeur de courant I_bat) pour notre exercice

  # Finalisation du réseau : on précise les paramètres d'apprentissage,  et
  modele.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate=taux_apprentissage),  # Le type d'optimiseur (RMSprop) et le taux d'apprentissage
                loss="mean_squared_error",                                                              # L'erreur à minimmiser (ici moyenne des distances au carré)
                metrics=[tf.keras.metrics.RootMeanSquaredError()])                                      # Une grandeur quantifiant la qualité de l'apprentissage
                                                                                                        # Ici c'est directement la racine carré de l'erreur
                                                                                                        # (pour ceux qui sont à l'aise, c'est la valeur efficace de la distance)
  return modele

On doit ensuite écrire la fonction qui permettra d'**entrainer le réseau**.  
- On dispose pour cela d'exemples de couples entrées/sorties "connus", que nous utiliserons comme **base d'apprentissage**. Pour nous, un couple c'est une de nos mesures, c'est à dire une valeur de tension U_bat mesurée pour un courant I_bat donné. La base d'apprentissage, c'est donc les 40 mesures (40 tensions U_bat pour 40 courants I_bat) présentées précédemment.  
- "**Entrainer**", cela signifie modifier les poids w et les biais b des neurones afin que pour une entrée donnée la sortie soit la meilleure possible. Le réseau modifie au fur et à mesure les poids/biais à partir des données entrées/sortie que nous lui fournissons en exemple afin qu'au final ça "colle" au mieux à tous ces exemples.  

La fonction d'entrainement a besoin de plusieurs arguments qu'il faut bien comprendre :

In [ ]:
# Fonction pour entrainer le modèle par régression linéaire
def entrainement(modele, feature, label, epochs, batch_size):
  # On fournit en arguments :
  # - modele : le réseau à entrainer
  # - feature : les "caractéristiques" d'entrée (x)
  # - label : les "étiquettes" de sortie (y)
  # - epochs : le nombre d'itérations de l'entrainement
  # - batch_size : le nombre de caractéristiques prises en compte dans une itération (entre 1 et le nb total disponible)
  # Par ex. si batch_size = 10, alors on utilise seulement les 10 premiers couples entrées/sorties de notre base d'apprentissage, puis les 10 suivants à l'itération 2...

  history = modele.fit(x=feature,     # Fonction TensorFlow d'entrainement du réseau
                      y=label,
                      batch_size=batch_size,
                      epochs=epochs)

  # Récupération des poids et biais calculés pour notre neurone
  trained_weight = modele.get_weights()[0]
  trained_bias = modele.get_weights()[1]

  # Récupération de la liste des itérations
  epochs = history.epoch

  # Récupération de l'erreur pour chaque itération
  rmse = pd.DataFrame(history.history)["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse

On définit les paramètres de l'entrainement avant de le lancer :

In [ ]:
taux_apprentissage=0.01   # "Vitesse" d'apprentissage. Par défaut 0.01
epochs=400                # Nombre d'itérations pour l'entrainement. Par défaut 400
batch=10                  # Nombre de caractéristiques prises en compte dans une itération. Par défaut 10

# Construction du réseau...
mon_modele = construction(taux_apprentissage)

# ... puis entrainement : à chaque itération il modifie les poids/biais pour que le modèle s'améliore
# Notez ici que l'on n'a pas fourni de direction de descente !
trained_weight, trained_bias, epochs, rmse = entrainement(mon_modele, I_bat,
                                                         U_mes, epochs,
                                                         batch)

#Affichage de l'évolution de l'erreur
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (epochs,rmse, linestyle="-", linewidth=1, label="Erreur")
ax.set_xlabel("Itérations")
ax.set_ylabel("Valeur de l'erreur")
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
plt.show()



Regardons ce que donne notre modèle pour l'intervale de courant ayant servi aux mesures :

In [ ]:
# Analyse du modèle obtenu pour notre réseau après apprentissage
U_mod = trained_bias + (I_bat * trained_weight) # B + W * I_bat

# Affichage des résultats du modèle
# Affichage des résultats du modèle
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (I_bat, U_th, linestyle="--", linewidth=0.5, label="Tension parfaite")
ax.plot (I_bat, U_mes, marker="+", linestyle="none", linewidth=1, label="Tension réelle mesurée")
ax.plot (I_bat, U_mod[0], linestyle="-", linewidth=1, color="red", label="Droite optimisée")
ax.set_title("Résultats du réseau")
ax.set_xlabel("Entrées Courant I_bat")
ax.set_ylabel("Tension U_bat")
ax.legend()
plt.show()

print ("Les paramètres théoriques sont : E_th = " + str(E_th) + "V et Ri_th = " + str(Ri_th) + "ohms ")
print ("Les paramètres calculés par le modèle en " + str(len(epochs)) + " itérations sont E_mod = " + str(trained_bias[0]) + "V et Ri_mod = " + str(-trained_weight[0][0]) + "ohms ")


La droite optimisée n'est pas si mauvaise, mais nous avons déjà obtenu des résultats similaires dans la partie 2... sauf qu'il faut se souvenir que là nous n'avons pas fourni l'expression de la pente : le réseau a appris tout seul, "sur le tas" !  
Le problème ici est aussi que les ordres de grandeurs des paramètres que l'on veut optimiser sont très différents : la valeur de la pente w est ~100x plus faible que le biais b car la résistance interne de la batterie est très faible. Les choses pourraient s'améliorer en normalisant les caractéristiques d'entrée, comme nous le verrons par la suite.  
Un autre point remarquable : il y a de l'aléatoire dans les résultats obtenus : comparez avec vos camarades, ou réentrainez le réseau et vous verrez que ce dernier obtiendra une autre droite !

Enfin, on teste généralement un réseau de neurones sur une base "non connue" (non utilisée pour l'apprentissage) afin de vérifier qu'il est capable de prédire une sortie pour une entrée qui n'a pas été apprise, par exemple ici pour un courant I_bat de 0.1 A, ou 10.6 A.  
Comment faire ces tests sur une entrée qui n'a pas servi à l'apprentissage ?

In [ ]:
# Prédiction sur des exemples non utilisés lors de l'apprentissage
x_test = [0.1, 1.4, 10.6]     # valeurs de I_bat courants tests
mon_modele.predict(x_test, verbose=0)

In [ ]:
# Prédiction sur un exemple particulier de la liste
choix = 2                     # valeur de I_bat à tester
print ("Notre modèle IA dit que la tension de sortie U_bat vaut " + str(mon_modele.predict(x_test, verbose=0)[choix][0]) + " V pour un courant de " + str(x_test[choix]) + " A")

Un neurone c'est sympa... mais ça limite un peu les choses ! => [Lien vers la partie 4 : Réseau de neurones artificiels](https://colab.research.google.com/drive/1dlNg6MqZYpc1VwPpx5fuTtkuDFD-Z3Fb#scrollTo=xYl_UBCdxmOC)

A COMPLETER AVEC PRISE EN COMPTE DE LA TEMPERATURE EN ENTREE 2  
+ Fonction activation RELU / Sigmoid ? A TESTER